In [48]:
import requests
import json
import csv,os,sys
from datetime import datetime
from bs4 import BeautifulSoup


#content = requests.get(url)
#page = BeautifulSoup(open('Coronavirus (COVID19) _ JAMA Network.html'), "html.parser")

lancetpage = "https://www.thelancet.com/coronavirus"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
}
htmlpage = requests.get(lancetpage, headers=headers)

page = BeautifulSoup(htmlpage.text, "html.parser")



for entry in mydiv.find_all("h4", attrs={"class": "title"}):
    txt.append(entry.text)
    
for entry in mydiv.find_all("div", attrs={"class": "articleType"}):
    Type.append(entry.text)
    
for entry in mydiv.find_all("div", attrs={"class": "published-online"}):
    Pdate = entry.text.split('Published: ')[-1]
    Pdate = datetime.strptime(Pdate , '%b %d %Y')
    Dates.append(Pdate.strftime("%Y%m%d"))
    
for entry in mydiv.find_all("div", attrs={"class": "doi"}):
    
    ID.append(entry.text.split('/')[-1])
    link.append(entry.text)

    


In [49]:
def FormatAbstract (AB):

    Abstract = ''
    tempAbs = AB.split(' ')
    lastN=0
    newabs = []
    for N in range(30, len(tempAbs)+30, 30) :
        newabs.append(' '.join(tempAbs[lastN:N]))
        lastN= N

    Abstract = '\n'.join(newabs)

    return(Abstract)

In [50]:
txt = []
link = []
ID = []
AB = []
Dates = []
Type = []

mydiv = page.select('div.widget-body.body.body-regular')[0]

for entry in mydiv.find_all("h4", attrs={"class": "title"}):
    txt.append(entry.text)
    
for entry in mydiv.find_all("div", attrs={"class": "articleType"}):
    Type.append(entry.text)
    
for entry in mydiv.find_all("div", attrs={"class": "published-online"}):
    Pdate = entry.text.split('Published: ')[-1]
    Pdate = datetime.strptime(Pdate , '%B %d, %Y')
    Dates.append(Pdate.strftime("%Y%m%d"))
    
for entry in mydiv.find_all("div", attrs={"class": "doi"}):
    
    ID.append(entry.text.split('/')[-1])
    link.append(entry.text)

    

    


In [51]:
json_file = 'lancent.json'
dict_file = 'lancent_dict_file.json'


#read the stored dictionary file (dict_file) or create a new blank dictionary
if os.path.isfile(dict_file):
    jsondict = json.load(open(dict_file))
else:
    jsondict = {}    
    
    
for N in range(0, len(ID)) :
    if ID[N] not in jsondict.keys():
        jsondict[ID[N]] = {'ID' : ID[N] , 
                          'txt' : txt[N],
                          'link' : link[N],
                           'Dates' : Dates[N],
                           'Type' : Type[N]
                          }
jsondict


#Write the main json file
with open(json_file , 'w') as fp:   
    Output=[]
    for Key,Item in jsondict.items():
        Output.append(Item)
    json.dump(Output, fp)

#Write the exact dict as json file (easily read by the script)
with open(dict_file , 'w') as fp:   
    json.dump(jsondict, fp)

In [53]:
def MakeTemplate():
    headerslist = []
    for H in ('ID' ,'txt'  ,'link' ,'Dates' ,'Type' ):

        print(H)
        headers = '=ImportJSON("https://raw.githubusercontent.com/tofaquih/coronaPubGet/master/lancent.json", "/{}", "noInherit,noTruncate")'.format(H)
        headerslist.append(headers)

    headerslist
    with open('lancent_template.csv' ,'w'  , newline='' ) as fp:
        W = csv.writer(fp, delimiter=';')
        W.writerow(headerslist)
        

In [54]:
MakeTemplate()

ID
txt
link
Dates
Type
